# **Swyp Teddy Bear Club NFT Rating and Rewards Calculation**

## **1. Overview**

### **1.1 Purpose**
This document outlines the process for calculating the rankings and rewards for the Teddy Bear Club NFTs across Round 1 and Round 2. Since the ranking and reward structures differ between the two rounds, calculations are performed separately for each round and then combined to determine the total rewards, where applicable.

### **1.2 NFT Information**
Round 1:<br>
   - Policy Id: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87 <br>
   - Total NFTs: 804<br>
   - Tier System:<br>
      - Tier 1 - Rarity 1 - 15 : 28,000 TEDY
      - Tier 2 - Rarity 16 - 100 : 17,500 TEDY
      - Tier 3 - Rarity 101 - 250 : 12,600 TEDY
      - Tier 4 - Rarity 251 - 500 : 11,200 TEDY
      - Tier 5 - Rarity 501 - 804 : 10,500 TEDY
      

Round 2:<br>
   - Policy Id: da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c<br>
   - Total NFTs: 2,035<br>
   - Tier System:<br>
      - Tier 1 - Rarity 1 - 23 : 7,000 TEDY
      - Tier 2 - Rarity 24 - 221 : 6,000 TEDY
      - Tier 3 - Rarity 222 - 664 : 5,000 TEDY
      - Tier 4 - Rarity 665 - 1,327 : 4,500 TEDY
      - Tier 5 - Rarity 1,328 - 2,035 : 4,200 TEDY

---

### **1.3 Workflow Summary**
The workflow is divided into the following steps:

1. **Fetch Data**  
   - Load NFT data from JSON files.

2. **NFT Rank Calculation**  
   2.1 Assign point values based on trait rarity.  
   2.2 Compute overall rarity ranks for each NFT.

3. **NFT Rewards Calculation**  
   - Calculate rewards for each NFT based on its rank and the round-specific reward structure.

---

### **1.4 Necessary Open Statements**


In [1]:
open System.Text.Json
open FSharp.Reflection

# **2. Workflow**

## 1. Fetch Data
The metadata for the NFTs was scraped from the web and stored in JSON files. In this section, we will parse the JSON data to extract information about the NFTs and the occurrence of their traits. This data will serve as the foundation for calculating rankings and determining rewards in the subsequent sections.

---

**Types**:
The NFT and NFTTraits types are partial representations of the data, as only the fields relevant to ranking and reward calculations have been included.

- **NFT**: Represents an individual NFT, including its metadata and traits.

- **NFTTraits**: Captures occurrence counts of trait values across NFTs.

**Function**:
- **loadNFTs<'T>**: Loads and deserializes JSON data into a specified type:
    - Parameters: 
        - filePath (string) – the path to the JSON file.
    - Returns: 'T 

**Variables**:
- **nftRound1NFTS**: NFT list – NFTs from Round 1
- **nftRound2NFTS**: NFT list – NFTs from Round 2
- **nftRound1Traits**: NFTTraits – Trait counts for Round 1
- **nftRound2Traits**: NFTTraits – Trait counts for Round 2


In [3]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [4]:
let loadNFTs<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [16]:
let nftRound1NFTS = loadNFTs<NFT list> "./NFTR1.json"
let nftRound2NFTS = loadNFTs<NFT list> "./NFTR2.json"
let nftRound1Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT1Traits.json"
let nftRound2Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT2Traits.json"
let nftRound1Traits = List.head nftRound1Data
let nftRound2Traits = List.head nftRound2Data

## **2. NFT Rank Calculation** 
To calculate the points for each trait, we first divide the number of occurrences of that trait by the total number of NFTs in the round, which gives us the percentage of NFTs that have that trait. We then subtract this percentage from 100, creating a direct relationship between rarity and points so rarer traits get higher points. Finally, we sum the points for each trait across different attributes (such as Background, Bear, Clothes, etc.) to calculate the final rarity score for each NFT.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{value}}{\text{totalNfts}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} T_i$

---
### 2.1 Assign point values based on trait rarity.   

**Function**:
- *processNFTTraits*: Calculates rarity scores for each trait dynamically for a given NFTTraits record and total NFTs in the round.
    - *Parameters*:
        *nftTraits*: NFTTraits – The trait occurrence data for a round.
    - *totalNfts*: float – The total number of NFTs in the round.
    - Returns: Map<string, Map<string, float>>

**Variables**:
- *r1TotalNfts*: float – The total number of NFTs in Round 1 (804).
- *r2TotalNfts*: float – The total number of NFTs in Round 2 (2035).
- *r1ScoredTraits*: Map<string, Map<string, float>> – A map of scored trait percentages for Round 1 NFTs.
- *r2ScoredTraits*: Map<string, Map<string, float>> – A map of scored trait percentages for Round 2 NFTs.


In [6]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [ ]:
let processNFTTraits (nftTraits: NFTTraits) (totalNfts: float) =
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [17]:
let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

### *Trait Scores*:

### 2.2 Compute overall rairty ranks for each NFT. 

**Function**:
- *computeNFTRank*: Computes the rarity rank of NFTs based on their traits and scores.
    - *Parameters*:
        *nftList*: NFT list – A list of NFTs to rank.
        *nftTraitScores*: Map<string, Map<string, float>> – Trait scores computed above.

**Variables**:
- *rankedNFTsRound1*: Ranked list of Round 1 NFTs as (rank, assetName, encodedName, rarityScore).
- *rankedNFTsRound2*: Ranked list of Round 2 NFTs with the same structure as above.


In [ ]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [20]:
let rankedNFTsRound1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTsRound2 = computeNFTRank nftRound2NFTS r2ScoredTraits